### 1. 安裝並執行 Ollama

首先是到官網抓下安裝程式, 並且安裝。

In [1]:
!curl -fsSL https://ollama.ai/install.sh | sh

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


因為我們要用 API 的方式呼叫, 所以需要跑 Ollama Server, 這裡我們要求放在背景執行。

In [2]:
!nohup ollama serve &

nohup: appending output to 'nohup.out'


建議先把會用到的模型抓下來, 這裡以 Llama 3.2 示範, 預設是 Llama 3.2 3B 模型。

In [3]:
!ollama pull gemma3:4b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling aeda25e63ebd: 100% ▕▏ 3.3 GB                         
pulling e0a42594d802: 100% ▕▏  358 B                         
pulling dd084c7d92a3: 100% ▕▏ 8.4 KB                         
pulling 3116c5225075: 100% ▕▏   77 B                         
pulling b6ae5839783f: 100% ▕▏  489 B                         
verifying sha256 digest 
writing manifest 
success 


### 2. 用 OpenAI API 使用

因為 ChatGPT 大概是最早紅的大型語言模型, 因此許多大型語言模型, 都和 OpenAI API 相容, Ollama 也不例外。

In [4]:
import openai
from openai import OpenAI

本來是需要 OpenAI 金鑰, 但我們沒有真的要用 OpenAI 的服務, 金鑰就亂打一通就好。

In [5]:
api_key = "ollama"

如同一般 OpenAI API 打開 `client` 的方式, 只是這裡多了 API 服務的網址。注意在自己家 (事實上是 Google Colab 的機器), 預設服務 `port` 是 `11434`。

In [6]:
client = OpenAI(
    api_key=api_key,
    base_url="http://localhost:11434/v1"
)

### 6. 打造一個你的對話機器人 web app!

In [7]:
!pip install gradio

In [8]:
import gradio as gr

對話機器人 app 設定

In [9]:
title = "呷啥機器人"
system = """你是一個專門協助使用者解決「不知道要吃什麼」的選擇障礙的機器人。根據使用者口味喜好、飲食習慣以及心情，推薦合適的餐點。
      回應都像在地嚮導一樣的口氣，有熱情的推薦使用者，儘量不要超過二十個字，請用台灣習慣的中文來回應。"""
description = "你好，我是你的 AI 小助手呷啥機器人, 不知道要吃甚麼的話可以跟我聊哦 :)"
model = "gemma3:4b"

In [10]:
initial_messages = [{"role":"system",
             "content":system},
            {"role":"assistant",
            'content':description}]

In [11]:
state = gr.State(initial_messages)

In [12]:
def pipi(prompt, messages):
    messages.append({"role": "user", "content": prompt})
    chat_completion = client.chat.completions.create(
        messages=messages,
        model=model,
        )
    reply = chat_completion.choices[0].message.content
    messages.append({"role": "assistant", "content": reply})
    return messages, messages

In [13]:
chatbot = gr.Chatbot(type="messages")

In [14]:
with gr.Blocks(title=title) as demo:
    gr.Markdown(f"## 🤖 {title}\n{description}")
    chatbot = gr.Chatbot(type="messages")
    msg = gr.Textbox(label="輸入訊息")
    state = gr.State(initial_messages.copy())  # 務必用 copy()

    msg.submit(fn=pipi, inputs=[msg, state], outputs=[chatbot, state])

demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://74ec8964d88a12133b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://74ec8964d88a12133b.gradio.live
